In [1]:
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())


In [2]:
from app.engine import Survey, QuestionPro
from app.engine.ai import PAAMSupervisor
from langchain_core.messages import HumanMessage


USER_AGENT environment variable not set, consider setting it to identify your requests.


ImportError: no pq wrapper available.
Attempts made:
- couldn't import psycopg 'c' implementation: No module named 'psycopg_c'
- couldn't import psycopg 'binary' implementation: No module named 'psycopg_binary'
- couldn't import psycopg 'python' implementation: argument should be a str or an os.PathLike object where __fspath__ returns a str, not 'NoneType'

In [3]:
questionpro = QuestionPro(
    survey_id='12032803', 
    api_key='61d15ae9-8995-4f6b-88ac-3a66e752a3cc'
)
await questionpro.fetch_data()
data = questionpro.data
survey = Survey(data=data)
survey.initialize()


In [4]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model='gpt-3.5-turbo', streaming=True)
graph = PAAMSupervisor(llm, survey).initialize()


In [35]:
message = "Hello"

inputs = {"messages": [HumanMessage(content=message)]}

msgs = []

async for msg in graph.astream(inputs, stream_mode="values"):
    msgs.append(msg)

In [36]:
for m in msgs:
    print(m)

{'messages': [HumanMessage(content='Hello', additional_kwargs={}, response_metadata={}, id='ec017120-fb96-4ffe-baf2-e2692541ae74')]}
{'messages': [HumanMessage(content='Hello', additional_kwargs={}, response_metadata={}, id='ec017120-fb96-4ffe-baf2-e2692541ae74')], 'next': 'FINISH'}


In [34]:
for m in msgs:
    for msg in m["messages"]:
        print(msg.content)

Hello. How are you
Hello. How are you
Hello. How are you
Hello! I'm just a computer program, so I don't have feelings, but I'm here to help you with any questions you may have. How can I assist you today?
